# COMP2002 Main Assignment
Machine Learning and Evolutionary Computation

In [1]:
import pandas
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

## Part 1- Machine Learning

### 1.1 Data Preparation
The forest fire datasets are CSVs with the following fields:
<table><tr>
    <td>Day</td>
    <td>Month</td>
    <td>Year</td>
    <td>Temperature (degrees C)</td>
    <td>RH (Relative Humidity, %)</td>
    <td>WS (Wind speed, kmph)</td>
    <td>Rain (Total for day, mm)</td>
    <td>FFMC (Fine Fuel Moisture Code index)</td>
    <td>DMC (Duff Moisture Code index)</td>
    <td>DC (Drought Code index)</td>
    <td>ISI (Initial Spread Index)</td>
    <td>BUI (Buildup Index)</td>
    <td>FWI (Fire Weather Index)</td></tr></table>

In [2]:
# Read in forest fire datasets
ff_region1 = pandas.read_csv("datasets/AlgerianFF_Region1.csv")
ff_region2 = pandas.read_csv("datasets/AlgerianFF_Region2.csv")
# Split into inputs and targets
region1_inputs_unnormalised = ff_region1[["day", "month", "year", "Temperature", " RH", " Ws", "Rain ", "FFMC", "DMC", "DC", "ISI", "BUI"]]
region1_targets = ff_region1["FWI"]
region2_inputs_unnormalised = ff_region2[["day", "month", "year", "Temperature", " RH", " Ws", "Rain ", "FFMC", "DMC", "DC", "ISI", "BUI"]]
region2_targets = ff_region2["FWI"]

# Normalise the data so that all features are in range 0-1
scaler = MinMaxScaler(feature_range=(0,1))
region1_inputs = scaler.fit_transform(region1_inputs_unnormalised)
region2_inputs = scaler.fit_transform(region2_inputs_unnormalised)

### 1.2 Regression

In [3]:
# Random Forest Regressor
# Implement normally, then experiment with different techniques of splitting the data (incl. that one that splits it into n groups and swaps between training and testing on each)
RF_regressor = RandomForestRegressor()
RF_regressor.fit(region1_inputs, region1_targets)

NameError: name 'region1_normalised' is not defined